In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# UTKFace

https://github.com/yu4u/age-estimation-pytorch/tree/master


In [3]:
import gc 
import torch
import torch.nn as nn
from torch import cuda
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as torchdata
from torch.autograd import Variable
from torchsummary import summary

import numpy as np 
import cv2
import random
import matplotlib.pyplot as plt
import os
from tensorflow.keras.utils import to_categorical


%matplotlib inline

import math
import time
import os
import sys
import random
import copy
from collections import deque
import cv2
# from google.colab.patches import cv2_imshow

import numpy as np
import pandas as pd
import json
import argparse
import matplotlib.pyplot as plt
%matplotlib inline
%time

import time
import sklearn
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

import copy
import pickle
import torchvision
import pandas as pd
from torch.utils.data import Dataset
import sklearn.preprocessing as preprocessing
import torch.optim as optim

import pandas as pd
import glob
import seaborn as sns
import imageio

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Found GPU at: {}'.format(device))

!/opt/bin/nvidia-smi -L
!ps -aux|grep python

from datetime import date

today = !date +"%m%d_%y"
print(f"\nReal Time: {today}")
today = today[0]
print(f"today: {today}")


sys.path.append(f"/home/hjchris/repo/utkface-estimation")
sys.path.append(f"/home/hjchris/repo/pytorch-ssd-master")
from vision.utils.misc import Timer

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 7.39 µs
Found GPU at: cuda:0
/bin/bash: /opt/bin/nvidia-smi: No such file or directory
root        1027  0.0  0.0  39304 20304 ?        Ss    2021   0:00 /usr/bin/python3 /usr/bin/networkd-dispatcher --run-startup-triggers
root        1166  0.0  0.0 118120 22992 ?        Ssl   2021   0:00 /usr/bin/python3 /usr/share/unattended-upgrades/unattended-upgrade-shutdown --wait-for-signal
hjchris     2788  0.1  0.1 342440 150164 ?       Sl    2021  11:11 /home/hjchris/research/mitigating/.venv/bin/python /home/hjchris/research/mitigating/.venv/bin/jupyter-notebook --no-browser --port=8888
hjchris    16710  0.0  0.0 902444 94368 ?        Ssl   2021   0:31 /home/hjchris/research/mitigating/.venv/bin/python -m ipykernel_launcher -f /home/hjchris/.local/share/jupyter/runtime/kernel-86170f7f-479f-4d41-93c0-1f2fed830e35.json
hjchris   311305 15.3  4.1 28226848 5432180 ?    Ssl   2021 603:33 /home/hjchris/research/mitigating/.venv/bin/python -m 

# Log

In [4]:
import logging

file_item=1
logging_root = f'/home/hjchris/research/mitigating/chris/utk_ckpt/'
logging_folder = logging_root + f'{today}'
logging_filename = f"{today}-{file_item}.log"

if not os.path.isdir(logging_folder):
    os.mkdir(logging_folder)
    print(f"Created directory: {logging_folder}")

while os.path.exists(os.path.join(logging_folder, logging_filename)):
    file_item += 1
    logging_filename = f"{today}-{file_item}.log"

print(f"File Name: {logging_filename}")
print(f"logging filename: {logging_folder}/{logging_filename}")

# basicConfig = logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s' )
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

streamHandler = logging.StreamHandler(sys.stdout)
streamHandler.setFormatter(formatter)

fileHandler=logging.FileHandler("{0}/{1}".format(logging_folder, logging_filename))
fileHandler.setFormatter(formatter)

train_log = logging.getLogger("train")
train_log.setLevel(logging.INFO)
if train_log.hasHandlers(): 
    train_log.removeHandler(fileHandler)
    train_log.removeHandler(streamHandler)
train_log.addHandler(fileHandler)
train_log.addHandler(streamHandler)

log_dir = logging_root + 'evaluations'
inf_log = logging.getLogger("Inference")
inf_log.setLevel(logging.INFO)   

Created directory: /home/hjchris/research/mitigating/chris/utk_ckpt/0102_22
File Name: 0102_22-1.log
logging filename: /home/hjchris/research/mitigating/chris/utk_ckpt/0102_22/0102_22-1.log


# Dataloader

## Helper Functions

In [5]:
'''
    Main Script for Gender, Age, and Ethnicity identification on the cleaned UTK Dataset.
    The dataset can be found here (https://www.kaggle.com/nipunarora8/age-gender-and-ethnicity-face-data-csv)
    I implemented a MultiLabelNN that performs a shared high-level feature extraction before creating a 
    low-level neural network for each classification desired (gender, age, ethnicity).
'''

import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
# import argparse
# torch imports
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter

# sys.path.append(f"/home/hjchris/repo/utkface-estimation")
# from CustomUTK import UTKDataset
# from MultNN import TridentNN

def _xavier_init_(m: nn.Module):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight)

'''
    Add arguments for argparse
    Arguments:
     - epochs : (int) Number of epochs to train
     - lr : (float) Learning rate for the model
     - pre-trained : (bool) whether or not to load the pre-trained model
'''

'''
    Function to read in the data
    Inputs: None
    Outputs:
     - train_loader : Custom PyTorch DataLoader for training data from UTK Face Dataset
     - test_loader : Custom PyTorch DataLoader for testing UTK Face Dataset
     - class_nums : Dictionary that stores the number of unique variables for each class (used in NN)
'''

class UTKDataset(Dataset):
    ''' =================================================================
        Inputs:
            dataFrame : Pandas dataFrame
            transform : The transform to apply to the dataset
    ===================================================================== '''
    def __init__(self, dataFrame, transform=None):
        # read in the transforms
        self.transform = transform
        
        # Use the dataFrame to get the pixel values
        data_holder = dataFrame.pixels.apply(lambda x: np.array(x.split(" "),dtype=float))
        arr = np.stack(data_holder)
        arr = arr / 255.0
        arr = arr.astype('float32')
        arr = arr.reshape(arr.shape[0], 48, 48, 1)
        # reshape into 48x48x1
        self.data = arr
        
        # get the age, gender, and ethnicity label arrays
        self.age_label = np.array(dataFrame.bins[:])         # Note : Changed dataFrame.age to dataFrame.bins
        self.gender_label = np.array(dataFrame.gender[:])
        self.eth_label = np.array(dataFrame.ethnicity[:])
    
    # override the length function
    def __len__(self):
        return len(self.data)
    
    # override the getitem function
    def __getitem__(self, index):
        # load the data at index and apply transform
        data = self.data[index]
        data = self.transform(data)
        
        # load the labels into a list and convert to tensors
        labels = torch.tensor((self.age_label[index], self.gender_label[index], self.eth_label[index]))
        
        # return data labels
        return data, labels

def read_data(dataFrame_path, train_batch, attack=False, test_split = 0.2, attack_split = 0.2, log=print):
    # Read in the dataframe
    dataFrame = pd.read_csv(dataFrame_path, compression='gzip')

    # Construct age bins
    age_bins = [0,10,15,20,25,30,40,50,60,120]
    age_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    dataFrame['bins'] = pd.cut(dataFrame.age, bins=age_bins, labels=age_labels)

    # Split into training and testing
    train_dataFrame, test_dataFrame = train_test_split(dataFrame, test_size=test_split)
    if attack:
        train_dataFrame, train_dataFrame_attack = train_test_split(train_dataFrame, test_size = attack_split)
        test_dataFrame, test_dataFrame_attack = train_test_split(test_dataFrame, test_size = attack_split)
    

    # get the number of unique classes for each group
    class_nums = {'age_num':len(dataFrame['bins'].unique()), 'ethnicity_num':len(dataFrame['ethnicity'].unique()),
                  'gender_num':len(dataFrame['gender'].unique())}

    # Define train and test transforms
    train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomCrop(48),
        transforms.RandomRotation(10),
        transforms.Normalize((0.49,), (0.23,))
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.49,), (0.23,))
    ])

    # Construct the custom pytorch datasets
    train_set = UTKDataset(train_dataFrame, transform=train_transform)
    test_set = UTKDataset(test_dataFrame, transform=test_transform)
    if attack:
        train_set_attack = UTKDataset(train_dataFrame_attack, transform=train_transform)
        test_set_attack = UTKDataset(train_dataFrame_attack, transform=train_transform)

    # Load the datasets into dataloaders
    train_loader = DataLoader(train_set, batch_size=train_batch, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=128, shuffle=False)
    
    if attack:
        train_loader_attack = DataLoader(train_set_attack, batch_size=train_batch, shuffle=True)
        test_loader_attack = DataLoader(test_set_attack, batch_size=128, shuffle=False)

    # Sanity Check
    log(f"Train Loader:{len(train_loader)}(batch size:{train_batch}), Test Laoder:{len(test_loader)}")
    for X, y in train_loader:
        log(f'Shape of training X: {X.shape}')
        log(f'Shape of y: {y.shape}')
        break
        
    if attack: 
        log(f"[Attack] Train Loader:{len(train_loader_attack)}, Test Laoder:{len(test_loader_attack)}")
        for X, y in train_loader_attack:
            log(f'Shape of training X: {X.shape}')
            log(f'Shape of y: {y.shape}')
            break
        return train_loader, test_loader, train_loader_attack, test_loader_attack, class_nums
    else:
        return train_loader, test_loader, class_nums

# MaxEnt

In [6]:
class Maxtrain_Network:
    def __init__(self, nets, optimizers, alpha = None, sensitive_class = None, public_classes = None, log=print):
        
        self.log=log
        self.net = nets[0]                
        self.discriminator = nets[1]
        self.public = nets[2]
        
        self.optimizer = optimizers[0]
        self.discriminator_optimizer = optimizers[1]
        self.public_optimizer = optimizers[2]
        
        self.alpha = alpha
        self.sensitive_classes = sensitive_class
        self.public_classes = public_classes
        
        self.entropy_loss = EntropyLoss()
        self.softmaxlayer = nn.Softmax(dim=1)
        self.logsoftmax = nn.LogSoftmax(dim=1)
        self.nll_loss = nn.NLLLoss()
        self.mse_loss = nn.MSELoss()
        self.cross_entloss = nn.CrossEntropyLoss()

        self.use_cuda = torch.cuda.is_available()
        self.images = torch.zeros(0, 0, 0)
        self.boxes = torch.zeros(0, 0, 0)
        self.labels = torch.zeros(0, dtype=torch.long)
        self.sensitive_labels = torch.zeros(0, dtype=torch.long)
        self.public_labels = torch.zeros(0, dtype=torch.long)
        self.grad_norm = torch.tensor(1.0)


        if self.use_cuda:
            self.images=self.images.cuda()
            self.boxes=self.boxes.cuda()
            self.labels=self.labels.cuda()
            self.grad_norm=self.grad_norm.cuda()

            if self.discriminator:
                self.sensitive_labels = self.sensitive_labels.cuda()
                self.alpha = self.alpha.to(device) 
            
            if self.public:
                self.public_labels = self.public_labels.cuda()
        
        self.meters_reset()

    def meters_reset(self):
        self.running_loss = 0.0
        self.running_regression_loss = 0.0
        self.running_classification_loss = 0.0
        self.running_regression_loss_dis = 0.0
        self.running_discriminator_loss = 0.0
        self.report_grad_norm = 0.0
        self.running_ssd_loss = 0.0
        self.running_discriminator_entropy = 0.0
        self.running_public_loss = 0.0
        self.running_public_entropy = 0.0

    def online_find_labels(self, labels, target_classes=[15]):
        label_temp = torch.zeros(len(labels), dtype=torch.long)
        for j in range(len(label_temp)):
            label_temp[j] = (1 if any(items in labels[j].unique() for items in target_classes) else 0)
        return label_temp

    def compute_gradnorm(self):
        grad_sum = 0
        for param in list(filter(lambda p: p.grad is not None, self.net.parameters())):
            grad_sum += math.pow(param.grad.data.norm(), 2) 
        return math.sqrt(grad_sum)
    
    def call_param(self, net_header, name, print_out=False):
        all_param = [param.detach().clone() for param in net_header.parameters()] 
        if print_out: print(f"{name}:{all_param[0][0][0]}")
        return all_param

    def assert_param(self, p1, p2):
        assert len(p1) == len(p2)
        for i in range(len(p1)): assert torch.sum(p1[i] == p2[i]) == p1[i].numel()

    def grad_summation(self, grads):
        grad_sum = 0 
        for grad in grads:
            if grad != None: grad_sum += torch.sum(grad)
        return grad_sum

    def train_net(self, net, net_optim, net_input, net_labels):
        
        net_input = net(net_input)
        net_loss = self.cross_entloss(net_input, net_labels)

        # Backprop
        net_optim.optimizer.zero_grad()
        net_loss.backward()
        _, grad_norm, _ = net_optim.step_all()

        return net_loss, grad_norm

    

    def arl_train(self, train_loader, private_label, public_label, d_loc=None, privacy_flag=True, 
                  public_flag = False, arl_setting='maxent-arl', debug_steps=100, epoch=-1, verbose=False, original=True):
        
        ''' ===================================================================================
        Main Algorithm:
            1. Train encoder and decoder with discriminator loss
            2. Train discriminator
        ========================================================================================'''
        self.net.train(True)
        if privacy_flag: self.discriminator.train(True)
        if public_flag: self.public.train(True)
        self.meters_reset()

        for i, (X, y) in enumerate(train_loader):

            # Prepare data
            if i%20 == 0 and i: print(end='.')
            labels = y[:,0].to(device), y[:,1].to(device), y[:,2].to(device)
            X = X.to(device)

            # 1. Train Encoder + Predictor =============
            z = self.net(X, d_loc)
                    
            if public_flag:
                self.public_optimizer.optimizer.zero_grad()
                p_output = self.public(z)
                p_loss = self.cross_entloss(p_output, labels[public_label])
                
            # Loss =====================================
            if privacy_flag:
                d_output = self.discriminator(z)
                d_prob = self.softmaxlayer(d_output)
                
                # Calculate for private task entropy/log liklihood
                if arl_setting == 'maxent-arl':
                    entropy_loss = -self.entropy_loss(d_prob)
                elif arl_setting == 'ml-arl':
                    entropy_loss = self.nll_loss(torch.log(d_prob+1e-16), labels[private_label])
                else:
                    raise Exception("Invalid ARL.")
                s_loss = -entropy_loss
            
            # Combined loss to update SSD (+ public task)
            if privacy_flag:
                if public_flag:
#                     if original:
                        loss = p_loss
#                     else:
#                         loss = (1-self.alpha)*(p_loss) + self.alpha*s_loss
            else:
                loss = p_loss
            
            if original and public_flag:
                self.discriminator_optimizer.optimizer.zero_grad()
                d_output = self.discriminator(z.detach())
                d_loss = self.cross_entloss(d_output, labels[private_label]) 
                loss += d_loss 

            # Backprop ==================================
            self.optimizer.optimizer.zero_grad()  
            loss.backward()
            if public_flag: _, _, _ = self.public_optimizer.step_all()                
            enc_grad_norm, self.grad_norm, shrinkage = self.optimizer.step_all()
            
            if privacy_flag: 
                if original:
                    _, _, _ = self.discriminator_optimizer.step_all()
                else:
                    # 2. Train Discriminator ====================
                    self.discriminator_optimizer.optimizer.zero_grad()
                    d_loss, _ = self.train_net(net = self.discriminator, 
                                               net_optim = self.discriminator_optimizer,
                                               net_input = z.detach(), 
                                               net_labels = labels[private_label]
                                              )    
        
            # 3. Save and Show Results ==================
            self.running_loss += loss.item() / debug_steps
            self.report_grad_norm += self.grad_norm / debug_steps
            if privacy_flag: 
                self.running_discriminator_loss += d_loss.item() / debug_steps
                self.running_discriminator_entropy += entropy_loss.item() /debug_steps
                
            if public_flag:
                self.running_public_loss += p_loss.item() / debug_steps
            
            #  Show the average results
            if i and i % debug_steps == 0:
                self.log(f"Epoch:{epoch:02d}, Step:{i}, " +
                             f"Loss:{self.running_loss:.4f}, " +
                             f"ARL Loss:{self.running_discriminator_entropy:.4f}, " + 
                             f"D Loss:{self.running_discriminator_loss:.4f}, " + 
                             f"P Loss:{self.running_public_loss:.4f}, " +
                             f"Grad Norm:{self.report_grad_norm:.4f}"
                            )
                self.meters_reset()              
    
    def test(self, test_loader, private_label, public_label, d_loc, privacy_flag = False, public_flag=False):
        
        self.net.eval()
        if privacy_flag: self.discriminator.eval()
        if public_flag: self.public.eval()
        self.meters_reset()
        num = 0
        self.net.test = True

        for i, (X,y) in enumerate(test_loader):

            num += 1
            if i%10 == 0: print(end='.')
            labels = y[:,0].to(device), y[:,1].to(device), y[:,2].to(device)
            X = X.to(device)

            # Sensitive label
            
            #  Feedforward
            with torch.no_grad():
                z = self.net(X, d_loc)
                
                if privacy_flag:
                    d_output = self.discriminator(z)
                    d_prob = self.softmaxlayer(d_output)
                    entropy_loss = -self.entropy_loss(d_prob)
                    d_loss = self.nll_loss(torch.log(d_prob+1e-16), labels[private_label])
                    
                if public_flag:
                    p_output = self.public(z)
                    p_prob = self.softmaxlayer(p_output)
                    p_entropy_loss = -self.entropy_loss(p_prob)
                    p_loss = self.nll_loss(torch.log(p_prob+1e-16), labels[public_label])
                    loss = p_loss

            #  Save numerical results
            self.running_loss += loss.item()
            if privacy_flag:
                self.running_discriminator_loss += d_loss.item()
                self.running_discriminator_entropy += entropy_loss.item()
            if public_flag:
                self.running_public_loss += p_loss.item()

        self.net.test = False

        return (self.running_loss/num, self.running_discriminator_loss/num, self.running_discriminator_entropy/num, self.running_public_loss/num)
    
    
    def arl_attack(self, train_loader, private_label, public_label, attacker_loc=None, privacy_flag=True, 
                   public_flag = False, debug_steps=40, epoch=-1, verbose=False):
        ''' ===================================================================================
        Main Algorithm:
            1. Train encoder and decoder with discriminator loss
            2. Train discriminator
        ======================================================================================== '''
        self.net.eval()
        self.discriminator.train(True)
        self.meters_reset()

        for i, (X, y) in enumerate(train_loader):

            # Prepare data
            if i%10 == 0 and i: print(end='.')
            labels = y[:,0].to(device), y[:,1].to(device), y[:,2].to(device)
            X = X.to(device)

            # 1. Train Encoder + Predictor =============
            with torch.no_grad():
                z = self.net(X, attacker_loc)
                
                if public_flag:
                    p_output = self.public(z)
                    p_prob = self.softmaxlayer(p_output)
                    p_entropy_loss = -self.entropy_loss(p_prob)
                    p_loss = self.cross_entloss(p_output, labels[public_label])
                
            if privacy_flag: 
                self.discriminator_optimizer.optimizer.zero_grad()
                d_loss, grad_norm = self.train_net(net = self.discriminator, 
                                           net_optim = self.discriminator_optimizer,
                                           net_input = z.detach(), 
                                           net_labels = labels[private_label])    

            # 3. Save and Show Results ==================
            self.running_loss += d_loss.item() / debug_steps
            self.report_grad_norm += grad_norm / debug_steps
            if public_flag:
                self.running_public_loss += p_loss.item() / debug_steps
                self.running_public_entropy += p_entropy_loss / debug_steps
            
            #  Show the average results
            if i and i % debug_steps == 0:
                self.log(f"Epoch:{epoch:02d}, Step:{i:3d}, " + 
                         f"Attacker Loss:{self.running_loss:.4f}, " +
                         f"P Loss:{self.running_public_loss:.4f}, " +
                         f"P Ent: {self.running_public_entropy:.4f}, " +
                         f"Grad Norm:{self.report_grad_norm:.4f}"
                         )
                self.meters_reset()

                # for param in net.classification_headers.parameters():
                #     print(f"Classification:{param[0][0]}")
                #     break                
        
    def test_attack(self, criterion, attacker_loc, privacy_flag = True):
        
        self.net.eval()
        self.discriminator.eval()
        self.meters_reset()
        num = 0
        self.net.test = True

        for i, (X,y) in enumerate(test_loader):

            num += 1
            if i%10 == 0: print(end='.')
            self.images.resize_(data[0].size()).copy_(data[0])
            self.boxes.resize_(data[1].size()).copy_(data[1])
            self.labels.resize_(data[2].size()).copy_(data[2])

            # Sensitive label
            sensitive_labels = data[3] if len(data) == 4 else self.online_find_labels(self.labels, self.sensitive_classes)
            self.sensitive_labels.resize_(sensitive_labels.size()).copy_(sensitive_labels)   
            
            #  Feedforward
            with torch.no_grad():
                confidence, locations, z = self.net(self.images, attacker_loc)
                d_output = self.discriminator(z)
                d_prob = self.softmaxlayer(d_output)
                d_loss = self.nll_loss(torch.log(d_prob+1e-16), self.sensitive_labels)
                entropy_loss = -self.entropy_loss(d_prob)
                s_loss = -entropy_loss 

                #  Loss
                class_loss, reg_loss = criterion(confidence, locations, self.labels, self.boxes)
                loss = class_loss + reg_loss

            #  Save numerical results
            self.running_loss += loss.item()
            self.running_regression_loss += reg_loss.item()
            self.running_classification_loss += class_loss.item()
            self.running_discriminator_loss += d_loss.item()
            self.running_discriminator_entropy += entropy_loss.item()

                 

        self.net.test = False

        return self.running_loss/num, self.running_regression_loss/num, self.running_classification_loss/num, self.running_discriminator_loss/num, self.running_discriminator_entropy/num
    

# Encoder, Discriminator

In [7]:
'''
    Code for a Multi Label Neural Network
    The Initial (Shared) layers are high-level feature extractors using convolutions.
    The three lower level feature extractors consist of another convolutional layer followed by a few Dense layers.
'''
import torch
from torch import nn
import torch.nn.functional as F

# High level feature extractor network (Adopted VGG type structure)
class highLevelNN(nn.Module):
    def __init__(self):
        super(highLevelNN, self).__init__()
        self.CNN = nn.Sequential(
            # first batch (32)
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.ReLU(),

            # second batch (64)
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.ReLU(),

            # Third Batch (128)
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
        )

    def forward(self, x, d_loc):
        out = self.CNN(x)
        return out

        
    def save(self, privacy_flag, public_flag, alpha, max_grad_norm, epoch, optimizers, schedulers, val_loss, val_d_ent, val_d_loss, 
             sensitive_classes, model_path, discriminator_loc=None, discriminator=None, public=None, val_p_loss=None, public_classes=None,
             arl_setting = None):
        
        dic = {'privacy_flag': privacy_flag, 
               'public_flag': public_flag,
               'epoch': epoch,
               'model_state_dict': self.state_dict(),
               'max_grad_norm': max_grad_norm, 
               'optimizer_state_dict': optimizers[0].optimizer.state_dict(),
               'scheduler_stat_dict': schedulers[0].state_dict() if schedulers[0] else None,
               'loss': val_loss,
              }
        
        if discriminator:
            dic['alpha'] = alpha
            dic['arl_setting'] = arl_setting
            dic['sensitive_classes'] = sensitive_classes
            dic['discriminator_loc'] = discriminator_loc
            dic['entropy'] = val_d_ent
            dic['discrimiantor_loss'] = val_d_loss
            dic['discriminator_optimizer_state_dict'] = optimizers[1].optimizer.state_dict()
            dic['discriminator_scheduler_state_dict'] = schedulers[1].state_dict() if schedulers[1] else None
            dic['discriminator_state_dict'] = discriminator.state_dict()
            
        if public_flag:
            dic['public_state_dict'] = public.state_dict()
            dic['public_optimizer_state_dict'] = optimizers[2].optimizer.state_dict()
            dic['public_scheduler_state_dict'] = schedulers[2].state_dict() if schedulers[2] else None
            dic['public_loss'] = val_p_loss
            dic['public_classes'] = public_classes
            
        torch.save(dic, model_path) 
        
    def save_attack(self, privacy_flag, public_flag, alpha, max_grad_norm, loss, epoch, optimizers, schedulers, attacker_loss, public_classes,
                    sensitive_classes, entropy, model_path, attacker_loc=None, public=None, attacker=None, name=None):
        
        dic = {'privacy_flag': privacy_flag, 
               'epoch': epoch,
               'model_state_dict': self.state_dict(),
               'max_grad_norm': max_grad_norm, 
               'loss': loss,
               'public_flag': public_flag
              }
        if public_flag:
            dic['public_classes'] = public_classes
            dic['public_state_dict'] = public.state_dict()
            dic['public_optimizer_state_dict'] = optimizers[2].optimizer.state_dict() if optimizers[2] else None
            dic['public_scheduler_state_dict'] = schedulers[2].state_dict() if schedulers[2] else None
        if privacy_flag:
            dic['alpha'] = alpha
            
        dic['attacked_model'] = name
        dic['sensitive_classes'] = sensitive_classes
        dic['attacker_loss'] = attacker_loss
        dic['attacker_loc'] = attacker_loc
        dic['attacker_optimizer_state_dict'] = optimizers[1].optimizer.state_dict()
        dic['attacker_scheduler_state_dict'] = schedulers[1].optimizer.state_dict() if schedulers[1] else None
        dic['attacker_state_dict'] = attacker.state_dict()
        dic['entropy'] = entropy
            
        torch.save(dic, model_path)

# Low level feature extraction module
class lowLevelNN(nn.Module):
    def __init__(self, num_out):
        super(lowLevelNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(in_features=2048, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=64)
        self.fc4 = nn.Linear(in_features=64, out_features=num_out)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3, stride=2, padding=1))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=3, stride=2, padding=1))
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x


class TridentNN(nn.Module):
    def __init__(self, num_age, num_gen, num_eth):
        super(TridentNN, self).__init__()
        # Construct the high level neural network
        self.CNN = highLevelNN()
        # Construct the low level neural networks
        self.ageNN = lowLevelNN(num_out=num_age)
        self.genNN = lowLevelNN(num_out=num_gen)
        self.ethNN = lowLevelNN(num_out=num_eth)

    def forward(self, x):
        x = self.CNN(x, 0)
        age = self.ageNN(x)
        gen = self.genNN(x)
        eth = self.ethNN(x)

        return age, gen, eth

## Entropy Loss

In [8]:
import torch
from  torch.nn.modules.loss import _Loss

class EntropyLoss(_Loss):
    def __init__(self, size_average=None, reduce=None, reduction='mean'):
        super(EntropyLoss, self).__init__(size_average, reduce, reduction)

    # input is probability distribution of output classes
    def forward(self, input):
        if (input < 0).any() or (input > 1).any():
            raise Exception('Entropy Loss takes probabilities 0<=input<=1')

        input = input + 1e-16  # for numerical stability while taking log
        H = torch.mean(torch.sum(input * torch.log(input), dim=1))
        
        return H
    
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.reshape(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

## Optimizer

In [9]:
class Optim(object):

    def _makeOptimizer(self):
        if self.method == 'sgd':
            self.optimizer = optim.SGD(self.params, lr=self.lr, momentum=self.sgd_momentum, weight_decay=self.sgd_weight_decay)
        elif self.method == 'adagrad':
            self.optimizer = optim.Adagrad(self.params, lr=self.lr)
        elif self.method == 'adadelta':
            self.optimizer = optim.Adadelta(self.params, lr=self.lr)
        elif self.method == 'adam':
#             self.optimizer = optim.Adam(self.params, lr=self.lr)
            self.optimizer = optim.Adam(self.params, lr=self.lr, betas=(self.adam_momentum, 0.999))
        else:
            raise RuntimeError("Invalid optim method: " + self.method)

    def __init__(self, params, method, enc_params = [0],  max_grad_norm = 5, lr = 0.001, lr_decay=1,
                 start_decay_at= 60, adam_momentum=0.9, sgd_momentum = 0.9, sgd_weight_decay = 5e-4, privacy_flag=False):
        
        self.params = params  # careful: params may be a generator
        self.encoder_params = list(enc_params)
        self.last_ppl = None
        self.lr = lr
        self.max_grad_norm = max_grad_norm
        self.method = method
        self.lr_decay = lr_decay
        self.start_decay_at = start_decay_at
        self.start_decay = False
        self.adam_momentum = adam_momentum
        self.sgd_momentum = sgd_momentum
        self.sgd_weight_decay = sgd_weight_decay
        self._makeOptimizer()

    def step_all(self):
        #  Calculate encoder gradient norm
        
        enc_grad_norm = 0.0
        for param in self.encoder_params:
            enc_grad_norm += math.pow(param.grad.data.norm(), 2) if param.grad != None else 0.0
        enc_grad_norm = math.sqrt(enc_grad_norm)

        #  Calculate all gradient norm
        grad_norm = 0.0
        for param in self.params:
            for p in param['params']:
                grad_norm += math.pow(p.grad.data.norm(), 2) 
        grad_norm = math.sqrt(grad_norm)

        #  Shrink the graident if the gradients are too large
        shrinkage = self.max_grad_norm / grad_norm
        for param in self.params:
            for p in param['params']:
                if shrinkage < 1:
                    p.grad.data.mul_(shrinkage)

        #  Start updating
        self.optimizer.step()
        
        return enc_grad_norm, grad_norm, shrinkage

# Training Setup

In [10]:
class TrainingSetup:
    def __init__(self, log=print, num_epochs=21, learning_rate = 1e-4, discriminator_lr = 1e-4, public_lr=1e-4, encoder_lr=1e-4):

        self.log = log
        
        # network hyperparameters
        self.learning_rate = learning_rate
        self.discriminator_lr = discriminator_lr
        self.public_lr = public_lr
        self.encoder_lr = encoder_lr
        self.base_net_lr =  self.learning_rate
        self.extra_layers_lr = self.learning_rate
        self.num_classes = 21
        self.scheduler = None
        
        # ARL setting
        self.privacy_flag = True
        self.public_flag = False
        self.attack_flag = False
        self.save = True
        self.saving_more = True
        self.continue_training = False

        # encoder setting
        self.public_headers = None
        self.public_optimizer = None
        self.public_scheduler = None
        
        self.discriminator_headers = None
        self.discriminator_optimizer = None
        self.discriminator_scheduler = None

        # network general setting        
        self.last_epoch = -1
        self.num_epochs = num_epochs
        self.loading_net = "basenet"
        self.scheduler_type = 'cosine'

        # optimizer setting
        self.opt_type = 'sgd'
        self.max_grad_norm = 6
        self.verbose = False
        self.t_max = 200
        self.momentum = 0.9
        self.weight_decay = 0.0005
        self.min_loss = -10000.0
        
        self.timer = Timer()

        if torch.cuda.is_available():
            torch.backends.cudnn.benchmark = True
            self.log("Use cuda:0")
    
    def set_directory(self):
        save_path, experiment_time = create_dir()
        self.log(f"="*45 + f" Training Session {today}-{experiment_time} " + f"="*45)
        
        if not os.path.exists(save_path): os.mkdir(save_path)
        self.checkpoint_folder = save_path
        
        if self.save: self.log(f"Model will be stored at {self.checkpoint_folder}")
    
    def build_discriminator(self, num_out):
        return lowLevelNN(num_out=num_out)
    
    def build_network(self):
        self.log("Build Network.")
        return highLevelNN()

    def build_init(self, load_path=None):
        timer = Timer()
        timer.start("Load Model")

    def build_scheduler(self, optimizer, name = 'Network', privacy_flag = False):
        
        if self.scheduler_type == "multi-step":
            self.log("Use MultiStepLR scheduler.")
            milestones = [int(v.strip()) for v in "80,100".split(",")]
            return MultiStepLR(optimizer, milestones=milestones, gamma=0.1, last_epoch=last_epoch)
        else:
            self.log(name + f" build [{self.scheduler_type}] CosineAnnealingLR scheduler. Max iter.={self.t_max}, last epoch={self.last_epoch}.")
            return CosineAnnealingLR(optimizer, self.t_max, last_epoch=self.last_epoch)

    def build_optim(self, parameters=None, name='Network', learning_rate=1e-4, opt_type = 'sgd'):
        self.log(name +  f" build {opt_type} optimizer in Optim, learning_rate = {learning_rate}, momentum={self.momentum}, " + 
                         f"weight_decay={self.weight_decay}, " + 
                         f"max gradient norm={self.max_grad_norm}.")   
 
        return Optim(params = [{'params': parameters,'lr': learning_rate}], 
                     method = opt_type, 
                     enc_params = [],
                     max_grad_norm = self.max_grad_norm, 
                     lr=learning_rate,
                     sgd_momentum=self.momentum,
                     sgd_weight_decay=self.weight_decay
                     )
    def build_rest(self):
        if not self.save: self.log(f"Not Saving model.")
            
    # ==========================================================================
    #                               Main Logic
    # ==========================================================================
    def build(self, load_path, privacy_flag, public_flag, prv_class_num, pub_class_num = 2, opt_type='adam', discriminator_opt_type='adam', is_attack=False):
        
        self.public_flag = public_flag
        self.privacy_flag = privacy_flag
        self.set_directory()
        
        # create Network [encoder and decoder]
        self.net = self.build_network()
        if not is_attack: self.optimizer = self.build_optim(parameters = self.net.parameters(),
                                                            learning_rate = 0.001,
                                                            opt_type=opt_type)
        
        # public task
        if self.public_flag:
            self.public_flag = public_flag
            self.public_headers = self.build_discriminator(num_out = pub_class_num)
            if not is_attack: self.public_optimizer = self.build_optim(name='Public',
                                                                       parameters = self.public_headers.parameters(),
                                                                       learning_rate = 0.001,
                                                                       opt_type = discriminator_opt_type)
            self.public_headers.to(device)
        
        # private task
        if self.privacy_flag:   
            build_name = 'Attacker' if is_attack else 'Discrimiantor'
            self.discriminator_headers = self.build_discriminator(num_out = prv_class_num)
            self.discriminator_optimizer = self.build_optim(name=build_name,
                                                            learning_rate = 0.001,
                                                            parameters = self.discriminator_headers.parameters(),
                                                            opt_type=discriminator_opt_type)
            self.discriminator_headers.to(device)
            
        # loading parameters
        self.build_rest()
        self.net.to(device)
        
        if is_attack:
            self._load_checkpoint(load_path)
        
    # ==========================================================================
    #                             Main Logic End
    # ==========================================================================
    
    def _load_checkpoint(self, load_path):
        self.log(f"Load the model from: {load_path}")
        self.ckpt = torch.load(load_path, map_location=lambda storage, loc: storage)
        self.net.load_state_dict(self.ckpt['model_state_dict'])
        
        self.privacy_flag = self.ckpt['privacy_flag']
        self.public_flag = self.ckpt['public_flag']
        self.max_grad_norm = self.ckpt['max_grad_norm']
        self.log(f"Privacy Flag={self.privacy_flag}, Public Flag={self.public_flag}, max_grad_norm={self.max_grad_norm}")
        
        if self.ckpt['privacy_flag']:
            self.sensitive_classes = self.ckpt['sensitive_classes']
            self.discriminator_loc = self.ckpt['discriminator_loc']
            self.alpha = self.ckpt['alpha']
            self.arl_setting = self.ckpt['arl_setting']
            self.log(f"alpha={self.alpha[0]}, sensitive_classes={self.sensitive_classes}, discriminator_loc={self.discriminator_loc}")

        if self.ckpt['public_flag']:
            self.public_classes = self.ckpt['public_classes']
            self.public_headers.load_state_dict(self.ckpt['public_state_dict'])
            
    
    def attack(self, train_loader, val_loader, attacker_loc, verbose=False, val_epoch=5, skip_train=False, previous_model=''): 
        ''' ====================================================================
        ARL Attack. 
        The discriminator is used as the attacker.
        
        
        ==================================================================== '''
        choice = ['age', 'gender', 'ethnicity']
        private_label = -1
        public_label = -1
        attack_model = previous_model
        
        self.log(f"Attacker Location={attacker_loc}. Attacked Model={previous_model}")
        
        if self.privacy_flag:
            self.log(f"Discriminator Location={self.discriminator_loc}, Alpha={self.alpha[0]}, Sensitive Classes={self.sensitive_classes}, ARL Setting={self.arl_setting}")
            private_label = choice.index(self.sensitive_classes)
        if self.public_flag:
            self.log(f"Public Classes = {self.public_classes}")
            public_label = choice.index(self.public_classes)
            
        
        self.log(f"Start attacking the model from epoch {self.last_epoch + 1}.") 
        
        nets = [self.net, self.discriminator_headers, self.public_headers]
        optimizers = [None, self.discriminator_optimizer, self.public_optimizer]
        schedulers = [None, None, None]

        train_net = Maxtrain_Network(nets, optimizers, self.alpha, self.sensitive_classes, log=self.log)
        
        
        for epoch in range(self.last_epoch + 1, self.num_epochs):
                                     
            if not skip_train:
                train_net.arl_attack(train_loader, 
                                     private_label = private_label, 
                                     public_label = public_label,
                                     privacy_flag = self.privacy_flag, 
                                     public_flag  = self.public_flag, 
                                     attacker_loc = attacker_loc, 
                                     debug_steps = 80,
                                     verbose=verbose,
                                     epoch=epoch)
            
            if epoch % val_epoch == 0 or epoch == self.num_epochs - 1 :                
                val_loss, val_d_loss, val_d_ent, val_p_loss = train_net.test(val_loader,
                                                                             private_label = private_label, 
                                                                             public_label = public_label,
                                                                             d_loc=attacker_loc,
                                                                             privacy_flag=self.privacy_flag,
                                                                             public_flag  = self.public_flag,
                                                                            )
                self.log(f"Epoch:{epoch:02d}, " +
                         f"Public Loss:{val_loss:.4f}, " +
                         f"Entropy:{val_d_ent:.4f}, " +
                         f"Attacker Loss:{val_d_loss:.4f}, "
                         )          
                #  Save
                if self.save:
                    model_name = f"{'utk-NN'}-attack-Ep{epoch:03d}-FM{attacker_loc}-Loss-{val_loss:.4f}+{val_d_loss:.4f}-Ent-{val_d_ent:.4f}.pth"
                    model_path = os.path.join(self.checkpoint_folder, model_name)
                    self.net.save_attack(privacy_flag=self.privacy_flag, 
                                         public_flag = self.public_flag,
                                         alpha = self.alpha, 
                                         max_grad_norm = self.max_grad_norm, 
                                         epoch = epoch,
                                         optimizers = optimizers,
                                         schedulers = schedulers, 
                                         loss = val_loss, 
                                         public = self.public_headers,
                                         sensitive_classes = self.sensitive_classes,
                                         public_classes = self.public_classes,
                                         entropy = val_d_ent,
                                         attacker = self.discriminator_headers,
                                         attacker_loss = val_d_loss, 
                                         attacker_loc = attacker_loc,
                                         model_path = model_path,
                                         name = attack_model
                                        )
                    self.log(f"Save models at: ~{model_path[48:]}")
                    if os.path.exists(previous_model): os.remove(previous_model)
                    previous_model = model_path
    
    
    def train(self, train_loader, val_loader, d_loc = 14, public_classes = [7], sensitive_classes=[15], alpha=1.0, verbose=False, val_epoch=5, 
              skip_train=False, previous_model='', arl_setting=None):
        ''' ======================================================================
        ARL Training
        
        ========================================================================== '''  
        choice = ['age', 'gender', 'ethnicity']
        private_label = -1
        public_label = -1
        
        if self.privacy_flag:
            self.log(f"Discriminator location={d_loc}, Alpha={alpha}, Sensitive classes={sensitive_classes}, Arl_setting={arl_setting}")
            private_label = choice.index(sensitive_classes)
        if self.public_flag:
            self.log(f"Public classes={public_classes}")
            public_label = choice.index(public_classes)
            
        self.log(f"Start training from epoch {self.last_epoch + 1}.") 
        
        nets = [self.net, self.discriminator_headers, self.public_headers]
        optimizers = [self.optimizer, self.discriminator_optimizer, self.public_optimizer]
        schedulers = [self.scheduler, self.discriminator_scheduler, self.public_scheduler]
        
        if self.privacy_flag:
            alpha = torch.tensor([alpha*1.0], requires_grad=True)

        train_net = Maxtrain_Network(nets, optimizers, alpha, sensitive_classes, public_classes, log=self.log)

        for epoch in range(self.last_epoch + 1, self.num_epochs):       
            
            if not skip_train:
                train_net.arl_train(train_loader,
                                    private_label = private_label, 
                                    public_label = public_label,
                                    privacy_flag = self.privacy_flag, 
                                    public_flag  = self.public_flag, 
                                    d_loc=d_loc,
                                    debug_steps=100, 
                                    epoch=epoch, 
                                    verbose=verbose,
                                    arl_setting = arl_setting,
                                   )

            # Evaluation
            if epoch % val_epoch == 0 or epoch == self.num_epochs - 1 :                

                val_loss, val_d_loss, val_d_ent, val_p_loss = train_net.test(val_loader,
                                                                             private_label = private_label, 
                                                                             public_label = public_label,
                                                                             d_loc=d_loc,
                                                                             privacy_flag=self.privacy_flag,
                                                                             public_flag  = self.public_flag,
                                                                            )
                self.log(f"Epoch:{epoch:02d}, " +
                         f"Validation Loss:{val_loss:.4f}, " +
                         f"Entropy:{val_d_ent:.4f}, " +
                         f"D Loss:{val_d_loss:.4f}, " +
                         f"P Loss:{val_p_loss:.4f}"
                         ) 
                
                if self.save:
                    model_name = f"{'utk-NN'}-Ep{epoch:03d}"
                    if self.privacy_flag:
                        model_name += f"-FM{d_loc}-A{alpha.data[0]:.1f}"
                    model_name += f"-Loss-{val_loss:.4f}"
                    if self.privacy_flag:
                        model_name += f"+{val_d_ent:.4f}-DL-{val_d_loss:.4f}"
                    if self.public_flag:
                        model_name += f"-PL-{val_p_loss:.4f}"
                    model_name += f".pth"
                        
                        
                    model_path = os.path.join(self.checkpoint_folder, model_name)
                    
                    self.net.save(privacy_flag = self.privacy_flag, 
                                  public_flag = self.public_flag,
                                  alpha = alpha, 
                                  max_grad_norm = self.max_grad_norm, 
                                  epoch = epoch, 
                                  optimizers = optimizers, 
                                  schedulers = schedulers, 
                                  val_loss = val_loss, 
                                  val_d_ent = val_d_ent, 
                                  val_d_loss = val_d_loss,
                                  sensitive_classes = sensitive_classes,
                                  discriminator_loc = d_loc,
                                  discriminator= self.discriminator_headers,
                                  model_path = model_path,
                                  public=self.public_headers,
                                  val_p_loss = val_p_loss,
                                  public_classes = public_classes,
                                  arl_setting = arl_setting
                                 )
                    
                    self.log(f"Save models at: ~{model_path[48:]}")
                    if os.path.exists(previous_model): os.remove(previous_model)
                    previous_model = model_path
                    

# Helper functions

In [11]:
import datetime

def create_dir():
    experiment_time = 1
    testing_dir = f"/home/hjchris/research/mitigating/chris/utk_ckpt/{today}/{today}-{experiment_time}"
    while (os.path.exists(testing_dir) and len([name for name in os.listdir(testing_dir) if os.path.isfile(os.path.join(testing_dir, name))]) != 0):
        experiment_time += 1
        testing_dir = f"/home/hjchris/research/mitigating/chris/utk_ckpt/{today}/{today}-{experiment_time}"
    
    return testing_dir, experiment_time


def write_doc(name, mode, dictionary, method=None, message=None):
    time = datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    with open("chris/utk_ckpt/document.txt", 'a') as file:
        file.write(f"{time} - File={name},\tTask={mode},\t")  
        
        for item in dictionary.items():
                file.write(f"{item[0]}={item[1]},\t ")
                if item[0] == 'privacy_flag' and item[1] == False: break
                if item[0] == 'public_flag' and item[1] == False: break
                
        if message:
                file.write("msg=(" + str(message) + ")\t")
        file.write("end\n")

        
def write_doc_end(name, mode):
    time = datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    with open("chris/utk_ckpt/document.txt", 'a') as file:
        file.write(f"{time} - File {name}   {mode} ends.\n")
        
        
class AverageMeter(object):
    """Computes and stores the average and current value
       Imported from https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.0
        self.avg = 0.0
        self.sum = 0.0
        self.count = 0.0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def find_ssd_file(checkpoint_folder):
    
    filenames=[]
    for filename in os.listdir(checkpoint_folder):
        if "pth" in filename:
            filenames.append(filename)
            
    ssd_path = checkpoint_folder + filenames[0]
    epoch = filenames[0].split("Ep")[1][:3]
    eval_path = checkpoint_folder + f"evaluation_{epoch}/"
    
    if not os.path.exists(eval_path):os.mkdir(eval_path)
        
    print(f"ssd path: {ssd_path}")
    print(f"eval path: {eval_path}")
    return ssd_path, eval_path

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataFrame_path = '/home/hjchris/repo/utkface-estimation/data/age_gender.gz'
train_loader, test_loader, train_loader_attack, test_loader_attack, class_nums = read_data(dataFrame_path, 
                                                                                           train_batch=64, 
                                                                                           attack=True, 
                                                                                           test_split = 0.2, 
                                                                                           attack_split = 0.4,
                                                                                           log=train_log.info)

2022-01-02 10:18:59,948 - train - INFO - Train Loader:178(batch size:64), Test Laoder:23
2022-01-02 10:19:00,017 - train - INFO - Shape of training X: torch.Size([64, 1, 48, 48])
2022-01-02 10:19:00,018 - train - INFO - Shape of y: torch.Size([64, 3])
2022-01-02 10:19:00,018 - train - INFO - [Attack] Train Loader:119, Test Laoder:60
2022-01-02 10:19:00,046 - train - INFO - Shape of training X: torch.Size([64, 1, 48, 48])
2022-01-02 10:19:00,047 - train - INFO - Shape of y: torch.Size([64, 3])


## run

In [13]:
message = None
private_flag_set = [True]
# arl_set = ['ml-arl']
arl_set = ['maxent-arl']
# arl_set = ['maxent-arl', 'ml-arl']

private_label = 'ethnicity'
public_label = 'gender'

attack_dates = ["1222"]
attack_nums = [9]
attack_locs = ["last"]
alphas = [0.5]

todays_task_is_train = True

if todays_task_is_train:
    for pv in private_flag_set:
        for arl_setting in arl_set:
            for alpha in alphas:
                train_dict ={'num_epochs': 21,
                             'test_split': 0.2, 
                             'attack_split': 0.4, 
                             'public_flag':True,
                             'public_label': public_label,
                             'pub_class_num': class_nums[f'{public_label}_num'],
                             'p_type': "A",
                             'privacy_flag':pv,
                             'method': arl_setting,
                             'alpha': alpha,
                             'private_label': private_label,
                             'prv_class_num': class_nums[f'{private_label}_num'],
                             'd_loc': 'last',
                             'd_type': "A",
                            }

                
                for rn in range(3):
                    arl_model = TrainingSetup(log=train_log.info,
                                              num_epochs=train_dict['num_epochs']
                                             )
                    gc.collect()
                    torch.cuda.empty_cache()
                    arl_model.build(load_path = 'a',
                                    prv_class_num = train_dict['prv_class_num'],
                                    pub_class_num = train_dict['pub_class_num'],
                                    public_flag = train_dict['public_flag'],
                                    privacy_flag = train_dict['privacy_flag']
                                    )

                    _, experiment_time = create_dir()
                    write_doc(f"{today}-{experiment_time}", mode="train", dictionary=train_dict, method=arl_setting, message=message)

                    arl_model.train(train_loader, 
                                    test_loader,
                                    arl_setting = arl_setting,
                                    alpha = train_dict['alpha'],
                                    d_loc = train_dict['d_loc'],
                                    verbose = True, 
                                    public_classes = public_label,
                                    sensitive_classes = private_label, 
                                    val_epoch = 5,
                                    skip_train = False
                                    )

                    write_doc_end(f"{today}-{experiment_time}", mode="train")


else:
    for att in range(min(len(attack_dates), len(attack_nums))):

        attack_date = attack_dates[att]
        attack_num = attack_nums[att]
        attack_loc = attack_locs[att]
        
        attack_root = '/home/hjchris/research/mitigating/chris/utk_ckpt/'
        attack_path, _ = find_ssd_file(attack_root + f"{attack_date}_21/{attack_date}_21-{attack_num}/")

        attack_dict ={'num_epochs': 101,
                      'attack_model':f"{attack_date}_21-{attack_num}", 
                      'attacker_loc': attack_loc,
                      'a_type': "A",
                      'public_label': public_label,
                      'pub_class_num': class_nums[f'{public_label}_num'],
                      'private_label': private_label,
                      'prv_class_num': class_nums[f'{private_label}_num'],
                     }

        arl_model = TrainingSetup(log=train_log.info,
                                  num_epochs=attack_dict['num_epochs']
                                             )

        gc.collect()
        torch.cuda.empty_cache()

        arl_model.build(attack_path, 
                        privacy_flag = True, 
                        public_flag = True, 
                        prv_class_num = attack_dict['prv_class_num'],
                        pub_class_num = attack_dict['pub_class_num'],
                        is_attack = True
                       )

        _, experiment_time = create_dir()
        write_doc(f"{today}-{experiment_time}", mode="attack", dictionary=attack_dict)

        arl_model.attack(train_loader_attack, 
                         test_loader_attack,
                         attacker_loc = attack_dict['attacker_loc'],
                         verbose=True, 
                         val_epoch=5,
                         previous_model = attack_dict['attack_model'],
                         skip_train=False
                       )
        
        write_doc_end(f"{today}-{experiment_time}", mode="attack")

2022-01-02 10:19:00,067 - train - INFO - Use cuda:0
2022-01-02 10:19:00,210 - train - INFO - ============================================= Training Session 0102_22-1 =============================================
2022-01-02 10:19:00,211 - train - INFO - Model will be stored at /home/hjchris/research/mitigating/chris/utk_ckpt/0102_22/0102_22-1
2022-01-02 10:19:00,212 - train - INFO - Build Network.
2022-01-02 10:19:00,216 - train - INFO - Network build adam optimizer in Optim, learning_rate = 0.001, momentum=0.9, weight_decay=0.0005, max gradient norm=6.
2022-01-02 10:19:00,230 - train - INFO - Public build adam optimizer in Optim, learning_rate = 0.001, momentum=0.9, weight_decay=0.0005, max gradient norm=6.
2022-01-02 10:19:05,424 - train - INFO - Discrimiantor build adam optimizer in Optim, learning_rate = 0.001, momentum=0.9, weight_decay=0.0005, max gradient norm=6.
2022-01-02 10:19:05,427 - train - INFO - Discriminator location=last, Alpha=0.5, Sensitive classes=ethnicity, Arl_sett

........2022-01-02 10:24:12,885 - train - INFO - Epoch:07, Step:100, Loss:1.3528, ARL Loss:1.0660, D Loss:1.0617, P Loss:1.3528, Grad Norm:1.3213
........2022-01-02 10:24:23,647 - train - INFO - Epoch:08, Step:100, Loss:1.3562, ARL Loss:1.0609, D Loss:1.0645, P Loss:1.3562, Grad Norm:1.3988
........2022-01-02 10:24:34,402 - train - INFO - Epoch:09, Step:100, Loss:1.2651, ARL Loss:0.9950, D Loss:0.9974, P Loss:1.2651, Grad Norm:1.2839
........2022-01-02 10:24:45,136 - train - INFO - Epoch:10, Step:100, Loss:1.2502, ARL Loss:0.9828, D Loss:0.9860, P Loss:1.2502, Grad Norm:1.3816
......2022-01-02 10:24:50,145 - train - INFO - Epoch:10, Validation Loss:0.2848, Entropy:1.0104, D Loss:0.9845, P Loss:0.2848
2022-01-02 10:24:50,241 - train - INFO - Save models at: ~/0102_22/0102_22-2/utk-NN-Ep010-FMlast-A0.5-Loss-0.2848+1.0104-DL-0.9845-PL-0.2848.pth
.....2022-01-02 10:24:56,385 - train - INFO - Epoch:11, Step:100, Loss:1.2251, ARL Loss:0.9823, D Loss:0.9728, P Loss:1.2251, Grad Norm:1.4556
..

2022-01-02 10:30:27,474 - train - INFO - Save models at: ~/0102_22/0102_22-3/utk-NN-Ep020-FMlast-A0.5-Loss-0.2953+0.8861-DL-0.9467-PL-0.2953.pth


# Inference 

In [ ]:
class Inference:
    def __init__(self, net_path, class_nums, log=print):
        
        self.log = log
        self.is_attack = True if "attack" in net_path else False
        
        self._load_checkpoint(net_path)
        self._load(class_nums)
    
    def _load(self, class_nums):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        gc.collect()
        torch.cuda.empty_cache()
        
        self.net = highLevelNN()
        self.net.load_state_dict(self.ckpt['model_state_dict'])
        
        if self.privacy_flag:
            self.discriminator = lowLevelNN(num_out=class_nums[f'{self.sensitive_classes}_num'])
            if self.is_attack:
                self.discriminator.load_state_dict(self.ckpt['attacker_state_dict'])
            else:
                self.discriminator.load_state_dict(self.ckpt['discriminator_state_dict'])
            self.discriminator.to(device)
            
        if self.public_flag:
            self.public = lowLevelNN(num_out=class_nums[f'{self.public_classes}_num'])
            self.public.load_state_dict(self.ckpt['public_state_dict'])
            self.public.to(device)
        
        self.net.to(device)
        
    def _load_checkpoint(self, net_path):
        
        print("Load checkpoint")
        self.ckpt = torch.load(net_path, map_location=lambda storage, loc: storage)
        self.epoch = self.ckpt['epoch']
        self.max_grad_norm = self.ckpt['max_grad_norm']
        self.loss = self.ckpt['loss']
        self.privacy_flag = self.ckpt['privacy_flag']
        self.public_flag = self.ckpt['public_flag']
        self.discriminator_loc = -10
        
        self.log(f"Epoch={self.epoch}, max_grad norm={self.max_grad_norm}, Loss={self.ckpt['loss']:.4f}")
        
        if self.privacy_flag:
            
            self.discriminator_loc = self.ckpt['discriminator_loc'] if not "attack" in ssd_path else self.ckpt['attacker_loc']
            self.sensitive_classes = self.ckpt['sensitive_classes']
            self.arl_setting = self.ckpt['arl_setting'] if not "attack" in ssd_path else "attack"
        
            total_loss = self.ckpt['total_loss'] if 'tota_loss' in self.ckpt.keys() else self.ckpt['loss']
            self.log(f"alpha={self.ckpt['alpha'][0]}, Sensitive Classes={self.ckpt['sensitive_classes']}, Total Loss={total_loss:.4f}, "+ 
                    f"method={self.arl_setting}")
        
        if self.public_flag:
            self.public_classes = self.ckpt['public_classes']
            self.log(f"Public classes={self.public_classes}") 
            
            
    def test(self, test_loader):
        
        choice = ['age', 'gender', 'ethnicity']
        private_count = 0
        public_count = 0
        total = 0
        self.net.eval()
        
        if self.privacy_flag: 
            self.discriminator.eval()
            private_label = choice.index(self.sensitive_classes)
        if self.public_flag: 
            self.public.eval()
            public_label = choice.index(self.public_classes)

        size = len(test_loader)
        
        for i, (X,y) in enumerate(test_loader):

            if i%10 == 0: print(end='.')
            labels = y[:,0].to(device), y[:,1].to(device), y[:,2].to(device)
            X = X.to(device)
            
            #  Feedforward
            with torch.no_grad():
                z = self.net(X, self.discriminator_loc)
                
                if self.privacy_flag:
                    d_output = self.discriminator(z)
                    private_count += (d_output.argmax(1) == labels[private_label]).type(torch.float).sum().item()
                    
                if self.public_flag:
                    p_output = self.public(z)
                    public_count += (p_output.argmax(1) == labels[public_label]).type(torch.float).sum().item()
                    
            total += len(y)
        
        if self.privacy_flag:
            self.log(f"Private Accuracy [{self.sensitive_classes}] : {private_count/total:.4f}")
        if self.public_flag:
            self.log(f"Public Accuracy [{self.public_classes}] : {public_count/total:.4f}")

In [ ]:
def create_Handler(log, log_dir, path_name):
    file_item=1
    logging_filename = f"{path_name} ({today}-{file_item}).log"

    if not os.path.isdir(log_dir):
        os.mkdir(log_dir)
        print(f"Created directory: {log_dir}")

    while os.path.exists(os.path.join(log_dir, logging_filename)):
        file_item += 1
        logging_filename = f"{path_name} ({today}-{file_item}).log"

    print(f"Log File = {log_dir}/{logging_filename}")

    if log.hasHandlers(): 
        log.removeHandler(inf_fileHandler)
        log.removeHandler(streamHandler)
    
    fileHandler = logging.FileHandler("{0}/{1}".format(log_dir, logging_filename))
    fileHandler.setFormatter(logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s" ))
    
    log.addHandler(fileHandler)
    log.addHandler(streamHandler)
    
    return fileHandler

## run

In [ ]:
inference_date = 1231
exp_nums = [5]
to_log = True

for exp_num in exp_nums:
    ssd_path, eval_path = find_ssd_file('/home/hjchris/research/mitigating/chris/utk_ckpt/' + f"{inference_date}_21/{str(inference_date)}_21-{str(exp_num)}/")

    if to_log: 
        inf_fileHandler = create_Handler(inf_log, log_dir, ssd_path.split('/')[-2])
    
    arl_inference = Inference(ssd_path, class_nums, log=inf_log.info)
    arl_inference.test(test_loader_attack)
    print()

In [ ]:
tridentNN = TridentNN(class_nums['age_num'], class_nums['gender_num'], class_nums['ethnicity_num'])
opt = torch.optim.Adam(tridentNN.parameters(), lr=0.001)

train(train_loader, tridentNN, opt, num_epoch=2, print_every=20, log=print)
print('Finished training, running the testing script...')
test(test_loader, tridentNN, log=print)